In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import math
import phe
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler
from phe import paillier
import bitstring
from bitstring import Bits
from secrets import token_hex
import copy
from Crypto.Cipher import DES
import pickle

%matplotlib inline

In [27]:
def cholesky_decomposition(A):
    d = A.shape[0]
    L = A.copy()
    #L = np.zeros(A.shape)
    for i in range(d-1):
        for j in range(i+1,d):
            L[i][j] = 0.0
    for j in range(d):
        for k in range(j-1):
            for i in range(j,d):
                L[i][j] -= (L[i][k]*L[j][k])
        L[j][j] = math.sqrt(L[j][j])
        for k in range(j+1,d):
            L[k][j] /= L[j][j]
    for i in range(d-1):
        for j in range(i+1,d):
            L[i][j] = 0.0
    return L

In [28]:
A = np.array([[ 6,  3,  4,  8],
       [ 3,  6,  5,  1],
       [ 4,  5, 10,  7],
       [ 8,  1,  7, 25]],dtype=np.float32)

In [29]:
L = cholesky_decomposition(A)

In [2]:
#L

In [3]:
#np.dot(L,L.T)

In [32]:
A = np.array([[ 6,  3,  4,  8],
       [ 3,  6,  5,  1],
       [ 4,  5, 10,  7],
       [ 8,  1,  7, 25]],dtype=np.float32)

In [33]:
L2 = np.linalg.cholesky(A)

In [4]:
#L2

In [5]:
#np.dot(L2,L2.T)

In [6]:
#A

In [37]:
def cholesky(A):
    """Performs a Cholesky decomposition of A, which must 
    be a symmetric and positive definite matrix. The function
    returns the lower variant triangular matrix, L."""
    n = len(A)

    # Create zero matrix for L
    L = np.zeros(A.shape)

    # Perform the Cholesky decomposition
    for i in range(n):
        for k in range(i+1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            
            if (i == k): # Diagonal elements
                # LaTeX: l_{kk} = \sqrt{ a_{kk} - \sum^{k-1}_{j=1} l^2_{kj}}
                L[i][k] = np.sqrt(A[i][i] - tmp_sum)
            else:
                # LaTeX: l_{ik} = \frac{1}{l_{kk}} \left( a_{ik} - \sum^{k-1}_{j=1} l_{ij} l_{kj} \right)
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - tmp_sum))
    return L

In [7]:
#cholesky(A)

In [39]:
def square_root(x):
    e = [pow(4,i) for i in range(int(x)) if pow(4,i)<=x][-1]
    r = 0
    #print(e)
    while e != 0:
        if x >= r+e:
            x -= (r+e)
            r = (r >> 1) +e
        else:
            r = (r >> 1)
        e = (e >> 2)
    return r

In [8]:
#square_root(100)

In [41]:
def poc_bitstring_class():
    binary = np.array(list(bin(255))[2:],dtype=np.bool)
    b = bitstring.Bits(float=1.23,length=32)
    print(b.bin)
    b = bitstring.Bits(float=-1.23,length=32)
    print(b.bin)
    
    a = bitstring.Bits(float=1.0,length=32)
    b = bitstring.Bits(float=-1.0,length=32)
    print(a.bin, b.bin,a.bin[:9])
    
    a = bitstring.Bits(float=1000.2090990909090909,length=32)
    b = bitstring.Bits(float=-127.0,length=32)
    print(a.bin, b.bin)
    
    a = bitstring.Bits(int=1,length=32)
    b = bitstring.Bits(int=-1,length=32)
    print(a.bin, b.bin)

In [9]:
#poc_bitstring_class()

In [43]:
def poc_paillier_encryption_scheme():
    pub,priv = paillier.generate_paillier_keypair()
    n = pub.encrypt(1.25)
    m = pub.encrypt(1.25)
    print(priv.decrypt(m+n))

In [10]:
#poc_paillier_encryption_scheme()

In [45]:
def poc_encryption_des():
    a = token_hex(4)
    b = token_hex(4)
    c = token_hex(4)
    
    print('Original Values : ')
    print('A : [ %s ], B : [ %s ], C : [ %s ]'%(a,b,c))
    print('--------------------------------------')
    print('First Encrypt with A and then B')
    des_in = DES.new(b, DES.MODE_ECB)
    des_in2 = DES.new(a,DES.MODE_ECB)
    c_cipher = des_in2.encrypt(des_in.encrypt(c))
    c_decrypted = des_in.decrypt(des_in2.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))
    
    print('--------------------------------------')
    print('First Encrypt with B and then A')
    des_in = DES.new(b, DES.MODE_ECB)
    des_in2 = DES.new(a,DES.MODE_ECB)
    c_cipher = des_in.encrypt(des_in2.encrypt(c))
    c_decrypted = des_in2.decrypt(des_in.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))
    
    print('--------------------------------------')
    print('First Encrypt with B and then B')
    des_in = DES.new(b, DES.MODE_ECB)
    c_cipher = des_in.encrypt(des_in.encrypt(c))
    c_decrypted = des_in.decrypt(des_in.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))
    
    print('--------------------------------------')
    print('First Encrypt with A and then A')
    des_in2 = DES.new(a,DES.MODE_ECB)
    c_cipher = des_in2.encrypt(des_in2.encrypt(c))
    c_decrypted = des_in2.decrypt(des_in2.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))

In [11]:
#poc_encryption_des()

In [49]:
##xt = np.zeros(user.temp.shape,dtype=np.object)
#for i in range(user.temp.shape[0]):
#    for j in range(user.temp.shape[1]):
#        b = Bits(float=user.temp[i][j],length=64)
#        print(b.bin)

In [2]:
def compute_twos_complement(binary):
    out = ''
    for i in binary:
        out += '1' if i=='0' else '0'
    #print(out)
    return Add()(out,((len(binary)-1)*'0')+'1')
        
def float_to_bin(num,size=24):
    twos_complement = True if num<0 else False
    int_p,frac_p = str(num).split('.')
    frac_p = float('0.'+frac_p)
    int_b = bin(int(int_p))[2:]
    frac_b = '.'
    for i in range(8):
        #print(frac_p)
        frac_p = frac_p*2
        t_i, t_f = str(frac_p).split('.')
        frac_b += t_i
        if t_i == '1':
            frac_p -= 1.
        #print(frac_p)
    out = int_b+frac_b
    out = out.replace('.','')
    #print(out)
    out = out[:-4]  ## Need to change to -8 in future
    out += '11111100'
    out = out.zfill(size)
    if twos_complement:
        temp_out = compute_twos_complement(out[:size-8])
    out = temp_out+out[size-8:] if twos_complement else out
    #print(out)
    return out

def int_to_bin(num):
    twos_complement = True if num<0 else False
    int_b = bin(int(num))[2:]
    if twos_complement:
        int_b = compute_twos_complement(int_b)
    return int_b

def bin_to_float(binary):
    int_p , frac_p = binary[:len(binary)-8], binary[len(binary)-8:]
    int_part = 0.0
    for i,j in enumerate(reversed(int_p[1:])):
        int_part += (int(j) * pow(2, i))
    #print(int_part)
    int_part -= int(binary[0])*pow(2,len(binary)-9) ## 15
    #print(int_part)
    pow_part = 0.0
    for i,j in enumerate(reversed(frac_p[1:])):
        pow_part += (int(j) * pow(2, i))
    pow_part -= pow(2,7)
    #print(pow_part)
    return int_part*pow(2,pow_part)

def bin_to_int(binary):
    int_part = 0
    for i,j in enumerate(reversed(binary[1:])):
        int_part += (int(j) * pow(2, i))
    int_part -= int(binary[0])*pow(2,len(binary)-1)
    return int_part

In [3]:
class Gate(object):
    pass

class LeftShift(Gate):
    def __call__(self,a,n):
        return a[n:] + '0' * n

class RightShift(Gate):
    def __call__(self,a,n):
        return '0' * n + a[:-n]

class And(Gate):
    def __call__(self,a,b):
        return a and b

class Or(Gate):
    def __call__(self,a,b):
        return a or b

class Xor(Gate):
    def __call__(self,a,b):
        return 0 if (a==1 and b==1) or (a==0 and b==0) else 1

class Not(Gate):
    def __call__(self,a):
        t = {0:1,1:0}
        return t[a]
                
class Add(Gate):
    
    def add(self,a,b,carry):
        #xor = Xor()
        #and_g = And()
        #or_g = Or()
        o1 = Xor()(a,b)
        o2 = And()(a,b)
        add = Xor()(carry,o1)
        o4 = And()(carry,o1)
        cary = Or()(o4,o2)
        return str(add), cary
    
    def __call__(self,a,b):
        carry = 0
        out = ''
        for idx,i in reversed(list(enumerate(zip(a[:16],b[:16])))):
            o,carry = self.add(int(i[0]),int(i[1]),carry)
            out += o
        #print(out)
        out = ''.join(reversed(out))
        return out + a[16:]

class Subtract(Gate):
    
    def __call__(self,a,b):
        b2 = compute_twos_complement(b[:16])
        b2 = b2 + b[16:]
        return Add()(a,b2)

class Multiplication(Gate):

    def __call__(self,a,b):
        s = '0'*16
        m = copy.copy(a[:16])
        for i in reversed(b[:16]):
            if int(i) != 0:     # when digit is one, add the intermediary product
                s = Add()(s, m)
            m = LeftShift()(m, 1)  # shift one per digit in b
        #exponent = Add()(a[16:],b[16:])
        s = RightShift()(s,4) ## Shifting left by 4 places to get exponent back to 2^-4

        return s+a[16:]
        
class Division(Gate):
    
    def __call__(self,a, b):
        reminder,reminder_prev = Subtract()(a[:16], b[:16]), copy.copy(a[:16])
        quotient = Add()('0'*len(a[:16]), ((len(a[:16])-1)*'0')+'1')
        while '1' in reminder:
            reminder_prev = copy.copy(reminder)
            reminder = Subtract()(reminder, b[:16])
            #print(bin_to_int(reminder))
            #quotient = Add()(quotient, '00000001')
            if (bin_to_int(reminder) < 0 and bin_to_int(reminder_prev) > 0) or \
                (bin_to_int(reminder) > 0 and bin_to_int(reminder_prev) < 0):
                break
            quotient = Add()(quotient, ((len(a[:16])-1)*'0')+'1')
        quotient = LeftShift()(quotient,4)
        return quotient+a[16:]
    
class Sqrt(Gate):
    
    def __call__(self,x):
        num = copy.copy(x[:16])
        xx = bin_to_int(x[:16])
        #e = [i for i in range(int(xx)) if pow(4,i)<=xx][-1]
        e = [i for i in range(int(xx)) if pow(4,i)<=xx]
        #e = e[-1] if e else 0
        e = int_to_bin(pow(4,e[-1])) if e else '0'*len(num)
        if len(e) < len(num):
            e = PadZerosLeft()(e,len(num) - len(e))
        r = '0'*len(e)
        while '1' in e:
            if GreaterThanEqualTo()(num, Add()(r,e)):
                num = Subtract()(num, Add()(r,e))
                r = Add()(RightShift()(r, 1), e)
            else:
                r = RightShift()(r, 1)
            e = RightShift()(e, 2)
        r = LeftShift()(r,2)
        return r + x[16:]
    
class GreaterThanEqualTo(Gate):
    def __call__(self,a,b):
        return bin_to_int(a) >= bin_to_int(b)
    
class PadZerosLeft(Gate):
    def __call__(self,a,n):
        return n*'0' + a 

class PadZerosRight(Gate):
    def __call__(self,a,n):
        return a + n*'0'
    
class GarbledCircuit(object):
    def __init__(A,b):
        self.A = A
        self.b = b
        
    def generate_beta(self):
        pass

In [4]:
def poc_AddSubtractDivMultiSqrt():
    add = Add()
    print(add('010','011'))
    print(Add()('010','011'))
    print(Subtract()('010','001'))
    print(Subtract()('111','010'))
    print(Add()('111','010'))
    print(bin_to_float(Add()(float_to_bin(1.5),float_to_bin(1.5))))
    print(bin_to_float(float_to_bin(1.5)))
    print(bin_to_float(float_to_bin(1.8)))
    print(compute_twos_complement('0001'))
    print(Multiplication()('1110','0001'))
    print(Sqrt()('00010000'))
    print(Sqrt()('00010000'))
    print(Sqrt()('000100'))
    print(Sqrt()('00100000'))
    print(Sqrt()('0101'))
    print(Division()('00000111','00000011')) 
    print(Division()('10000000','10000000'))
    print(Division()('10000000','00000001'))
    print(Division()('00000001','11111111')) ## Wrong
    #print(Subtract()('10000000','00000001'))

In [56]:
#poc_AddSubtractDivMultiSqrt()

In [57]:
'https://www.electrical4u.com/2s-complement-arithmetic/'

'https://www.electrical4u.com/2s-complement-arithmetic/'

In [132]:
def cholesky_decomposition_binary(A):
    d = A.shape[0]
    L = A.copy()
    #L = np.zeros(A.shape)
    for i in range(d-1):
        for j in range(i+1,d):
            L[i][j] = '0'*24
    for j in range(d):
        for k in range(j-1):
            for i in range(j,d):
                out = Multiplication()(L[i][k], L[j][k])
                L[i][j] = Subtract()(L[i][j], out) ## Need to bring -4 as exponent
        L[j][j] = Sqrt()(L[j][j]) ## Need to bring -4 as exponent
        for k in range(j+1,d):
            L[k][j] = Division()(L[k][j],L[j][j]) ## Need to bring -4 as exponent
    return L

In [16]:
#A2 = np.empty(A.shape,dtype=np.object)
#for i in range(A.shape[0]):
#    for j in range(A.shape[1]):
#        A2[i][j] = float_to_bin(A[i][j])

In [60]:
#A2

In [61]:
#A3 = np.empty(A2.shape,dtype=np.object)
#for i in range(A2.shape[0]):
#    for j in range(A2.shape[1]):
#        A3[i][j] = bin_to_float(A2[i][j])

In [62]:
#A,A3

In [5]:
%time ans = Add()('000000000110000011111100', '000000000011000011111100')
print(bin_to_int('0000000000110000'),bin_to_int('0000000001100000'),bin_to_int(ans[:16]))

%time ans = Subtract()(compute_twos_complement('000000000110000011111100'), compute_twos_complement('000000000011000011111100'))
print(bin_to_int('0000000001100000'),bin_to_int('0000000000110000'),bin_to_int(ans[:16]))

%time ans = Multiplication()('000000000110000011111100', '000000000011000011111100')
print(bin_to_float('000000000110000011111100'),bin_to_float('000000000011000011111100'),bin_to_float(ans))

%time ans = Division()('000000000110000011111100', '000000000011000011111100')
print(bin_to_float('000000000110000011111100'),bin_to_float('000000000011000011111100'),bin_to_float(ans))

%time ans = Sqrt()('000000000110000011111100')
print(bin_to_float('000000000110000011111100'),bin_to_float(ans))

CPU times: user 165 µs, sys: 33 µs, total: 198 µs
Wall time: 207 µs
48 96 144
CPU times: user 770 µs, sys: 0 ns, total: 770 µs
Wall time: 706 µs
96 48 -48
CPU times: user 471 µs, sys: 0 ns, total: 471 µs
Wall time: 412 µs
6.0 3.0 18.0
CPU times: user 1.53 ms, sys: 0 ns, total: 1.53 ms
Wall time: 1.06 ms
6.0 3.0 2.0
CPU times: user 2.66 ms, sys: 0 ns, total: 2.66 ms
Wall time: 2.28 ms
6.0 2.25


In [134]:
#%time L2 = cholesky_decomposition_binary(A2)

In [69]:
L3 = np.empty(L2.shape,dtype=np.object)
for i in range(L2.shape[0]):
    for j in range(L2.shape[1]):
        L3[i][j] = bin_to_float(L2[i][j])

In [17]:
#L3,cholesky(A),cholesky_decomposition(A)

In [71]:
np.dot(L3,L3.T), A

(array([[5.0625, 2.25, 2.25, 6.75],
        [2.25, 6.0625, 5.5, 2050.5],
        [2.25, 5.5, 14.0, 1826.0],
        [6.75, 2050.5, 1826.0, 828110.0]], dtype=object),
 array([[ 6.,  3.,  4.,  8.],
        [ 3.,  6.,  5.,  1.],
        [ 4.,  5., 10.,  7.],
        [ 8.,  1.,  7., 25.]], dtype=float32))

In [72]:
def back_substitution_lower_np(L,Y):
    beta = np.empty(Y.shape,dtype=np.float32)
    beta[0] = Y[0] / L[0][0]
    for i in range(1,len(beta)):
        s = 0.0
        for j in range(i):
            mul = L[i][j] * beta[j]
            # print(i,j,L[i][j],beta[j],mul)
            s += mul
        #print(Y[i] / s)
        beta[i] = Y[i] / s
    return beta

In [73]:
def back_substitution_garbled(L,Y):
    beta = np.empty(Y.shape,dtype=np.float32)
    beta[0] = Divide()(Y[0], L[0][0])
    for i in range(1,len(beta)):
        s = '0'*16 + '00000000'
        for j in range(i):
            mul = Multiplication()(L[i][j], beta[j])
            s = Add()(s, mul)
        beta[i] = Divide()(Y[i], s)
    return beta

In [74]:
%time back_substitution_lower_np(cholesky(A),np.array([1.,2.,3.,4.]))

CPU times: user 464 µs, sys: 32 µs, total: 496 µs
Wall time: 502 µs


array([ 0.4082483 ,  3.9999998 ,  0.47441927, -1.1203607 ], dtype=float32)

In [12]:
def bin_to_int_garbled(binary,wire_labels):
    wl = dict(zip(wire_labels.values(),wire_labels.keys()))
    b2 = ''.join([wl[i].split('_')[-1] for i in binary])
    int_part = 0
    for i,j in enumerate(reversed(b2[1:])):
        int_part += (int(j) * pow(2, i))
    int_part -= int(wl[binary[0]].split('_')[-1])*pow(2,len(binary)-1)
    return int_part

def compute_twos_complement_garbled(binary,wire_labels={}):
    wl = dict(zip(wire_labels.values(),wire_labels.keys()))
    out = []
    for i in binary:
        out += [wire_labels['B_1']] if '0' in wl[i] else [wire_labels['B_0']]
    #print(out)
    out = GarbledAdd()(((len(binary)-1)*[wire_labels['A_0']])+[wire_labels['A_1']], out, wire_labels)
    wl = dict(zip(wire_labels.values(),wire_labels.keys()))
    out = [wire_labels[wl[i].replace('C','B')] for i in out]
    return out

class GarbledLeftShift(Gate):
    def __call__(self,a, n, wire_labels={}, garbled_table=[]):
        #print(a,wire_labels)
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        val =  [j for j in [wl[i] for i in list(set(a))] if '0' in j][0]
        return a[n:] + [wire_labels[val]] * n

class GarbledRightShift(Gate):
    def __call__(self,a, n, wire_labels={}, garbled_table=[]):
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        val =  [j for j in [wl[i] for i in list(set(a))] if '0' in j][0]
        return [wire_labels[val]] * n + a[:-n]

class GarbledGreaterThanEqualTo(Gate):
    def __call__(self,a, b, wire_labels):
        return bin_to_int_garbled(a, wire_labels) >= bin_to_int_garbled(b, wire_labels)
    
class GarbledPadZerosLeft(Gate):
    def __call__(self,a, n, wire_labels={}, garbled_table=[]):
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        val =  [j for j in [wl[i] for i in list(set(a))] if '0' in j][0]
        return [wire_labels[val]] * n + a 

class GarbledPadZerosRight(Gate):
    def __call__(self,a, n, wire_labels={}, garbled_table=[]):
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        val =  [j for j in [wl[i] for i in list(set(a))] if '0' in j][0]
        return a + [wire_labels[val]] * n
    
class GarbledAnd(Gate):
    def __call__(self,a,b,wire_labels={},garbled_table=[]):
        
        des_b_0 = DES.new(wire_labels['B_0'], DES.MODE_ECB)
        des_a_0 = DES.new(wire_labels['A_0'], DES.MODE_ECB)
        des_b_1 = DES.new(wire_labels['B_1'], DES.MODE_ECB)
        des_a_1 = DES.new(wire_labels['A_1'], DES.MODE_ECB)
        ## 0 AND 0 = 0
        
        garbled_table.append(des_a_0.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_0.encrypt(des_b_1.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_1.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_1.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        
        des_in = DES.new(b, DES.MODE_ECB)
        des_out = DES.new(a, DES.MODE_ECB)
        for g in garbled_table:
            try:
                deciphered = des_in.decrypt(des_out.decrypt(g)).decode()
            except UnicodeDecodeError:
                continue
            if deciphered in [ wire_labels['C_0'], wire_labels['C_1'] ]:
                return g
        
class GarbledOr(Gate):
    def __call__(self,a,b,wire_labels={},garbled_table=[]):
        
        des_b_0 = DES.new(wire_labels['B_0'], DES.MODE_ECB)
        des_a_0 = DES.new(wire_labels['A_0'], DES.MODE_ECB)
        des_b_1 = DES.new(wire_labels['B_1'], DES.MODE_ECB)
        des_a_1 = DES.new(wire_labels['A_1'], DES.MODE_ECB)
        ## 0 AND 0 = 0
        
        garbled_table.append(des_a_0.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_0.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_0.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        
        des_in = DES.new(b, DES.MODE_ECB)
        des_out = DES.new(a, DES.MODE_ECB)
        for g in garbled_table:
            try:
                deciphered = des_in.decrypt(des_out.decrypt(g)).decode()
            except UnicodeDecodeError:
                continue
            if deciphered in [ wire_labels['C_0'], wire_labels['C_1'] ]:
                return g

class GarbledXor(Gate):
    def __call__(self,a,b,wire_labels={},garbled_table=[]):
        
        des_b_0 = DES.new(wire_labels['B_0'], DES.MODE_ECB)
        des_a_0 = DES.new(wire_labels['A_0'], DES.MODE_ECB)
        des_b_1 = DES.new(wire_labels['B_1'], DES.MODE_ECB)
        des_a_1 = DES.new(wire_labels['A_1'], DES.MODE_ECB)
        ## 0 AND 0 = 0
        
        garbled_table.append(des_a_0.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_0.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_0.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_1.encrypt(wire_labels['C_0'])))
        #print(garbled_table)
        des_1 = DES.new(a, DES.MODE_ECB)
        des_2 = DES.new(b, DES.MODE_ECB)
        for g in garbled_table:
            try:
                deciphered = des_2.decrypt(des_1.decrypt(g)).decode()
            except UnicodeDecodeError:
                continue
            #print(deciphered)
            #print([ wire_labels['C_0'], wire_labels['C_1'] ])
            if deciphered in [ wire_labels['C_0'], wire_labels['C_1'] ]:
                return g
        
class GarbledNot(Gate):
    def __call__(self,b,wire_labels={},garbled_table=[]):
        
        des_a_0 = DES.new(wire_labels['A_0'], DES.MODE_ECB)
        des_a_1 = DES.new(wire_labels['A_1'], DES.MODE_ECB)
        
        garbled_table.append(des_a_0.encrypt(wire_labels['C_1']))
        garbled_table.append(des_a_1.encrypt(wire_labels['C_0']))
        
        des = DES.new(b, DES.MODE_ECB)
        for g in garbled_table:
            try:
                deciphered = des.decrypt(g).decode()
            except UnicodeDecodeError:
                continue
            if deciphered in [ wire_labels['C_0'], wire_labels['C_1'] ]:
                return g
            
class GarbledAdd(Gate):
    
    def add(self,a,b,carry,wire_labels):
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        
        des_in = DES.new(b, DES.MODE_ECB)
        des_out = DES.new(a, DES.MODE_ECB)
        
        o1 = GarbledXor()(a,b,wire_labels)
        #print(a,b,o1)
        o1 = des_in.decrypt(des_out.decrypt(o1)).decode()
        
        o2 = GarbledAnd()(a,b,wire_labels)
        o2 = des_in.decrypt(des_out.decrypt(o2)).decode()
        
        o1 = wire_labels[wl[o1].replace('C','B')]
        
        des_in = DES.new(o1, DES.MODE_ECB)
        des_out = DES.new(carry, DES.MODE_ECB)
        
        add = GarbledXor()(carry,o1,wire_labels)
        add = des_in.decrypt(des_out.decrypt(add)).decode()
        
        o4 = GarbledAnd()(carry,o1,wire_labels)
        o4 = des_in.decrypt(des_out.decrypt(o4)).decode()
        
        o4 = wire_labels[wl[o4].replace('C', 'A')]
        o2 = wire_labels[wl[o2].replace('C', 'B')]
        
        des_in = DES.new(o2, DES.MODE_ECB)
        des_out = DES.new(o4, DES.MODE_ECB)
        
        cary = GarbledOr()(o4, o2, wire_labels)
        cary = des_in.decrypt(des_out.decrypt(cary)).decode()
        
        cary = wire_labels[wl[cary].replace('C', 'A')]
        return str(add), cary
    
    def __call__(self,a, b, wire_labels={}):
        wl = dict(zip(wire_labels.values(),wire_labels.keys()))
        carry = wire_labels['A_0']
        out = []
        for idx,i in reversed(list(enumerate(zip(a[:16],b[:16])))):
            o,carry = self.add(i[0], i[1], carry, wire_labels)
            out.append(o)
        #print(out)
        out = list(reversed(out))
        return out + [ wire_labels[wl[i].replace('A','C')] for i in a[16:]]

class GarbledSubtract(Gate):
    
    def __call__(self,a,b, wire_labels={}):
        b2 = compute_twos_complement_garbled(b[:16],wire_labels)
        #print(b2,b[16:])
        b2 = list(b2) + list(b[16:])
        return GarbledAdd()(a,b2,wire_labels)

class GarbledMultiplication(Gate):

    def __call__(self,a,b, wire_labels={}):
        wl = dict(zip(wire_labels.values(),wire_labels.keys()))
        s = [wire_labels['B_0']]*16
        m = copy.copy(a[:16])
        for i in reversed(b[:16]):
            #print(m,s)
            if '1' in wl[i]:     # when digit is one, add the intermediary product
                s = GarbledAdd()(m, s, wire_labels)
                s = [ wire_labels[wl[i].replace('C','B')] for i in s]
            m = GarbledLeftShift()(m, 1, wire_labels)  # shift one per digit in b
        #exponent = Add()(a[16:],b[16:])
        s = [ wire_labels[wl[i].replace('B','C')] for i in s]
        s = GarbledRightShift()(s,4,wire_labels) ## Shifting left by 4 places to get exponent back to 2^-4
        return s+[ wire_labels[wl[val].replace('B', 'C')] for val in b[16:]]
        
class GarbledDivision(Gate):
    
    def __call__(self,a, b,wire_labels={}):
        wl = dict(zip(wire_labels.values(),wire_labels.keys()))
        reminder,reminder_prev = GarbledSubtract()(a[:16], b[:16], wire_labels), copy.copy(a[:16])
        quotient = GarbledAdd()([wire_labels['A_0']]*16, ([wire_labels['B_0']] * 15)+[wire_labels['B_1']], wire_labels)
        reminder = [ wire_labels[wl[val].replace('C', 'A')] for val in reminder]
        while wire_labels['A_1'] in reminder:
            reminder_prev = copy.copy(reminder)
            reminder = GarbledSubtract()(reminder, b[:16], wire_labels)
            reminder = [ wire_labels[wl[val].replace('C', 'A')] for val in reminder]
            #print(bin_to_int(reminder))
            #quotient = Add()(quotient, '00000001')
            if (bin_to_int_garbled(reminder,wire_labels) < 0 and bin_to_int_garbled(reminder_prev,wire_labels) > 0) or \
                (bin_to_int_garbled(reminder,wire_labels) > 0 and bin_to_int_garbled(reminder_prev,wire_labels) < 0):
                break
            quotient = [ wire_labels[wl[val].replace('C', 'A')] for val in quotient]
            quotient = GarbledAdd()(quotient, ([wire_labels['B_0']] * 15)+[wire_labels['B_1']], wire_labels)
        quotient = GarbledLeftShift()(quotient,4, wire_labels)
        return quotient+[ wire_labels[wl[val].replace('A', 'C')] for val in a[16:]]
    
class GarbledSqrt(Gate):
    
    def __call__(self,x,wire_labels={}):
        wl = dict(zip(wire_labels.values(), wire_labels.keys()))
        num = copy.copy(x[:16])
        xx = bin_to_int_garbled(x[:16],wire_labels)
        #e = [i for i in range(int(xx)) if pow(4,i)<=xx][-1]
        e = [i for i in range(int(xx)) if pow(4,i)<=xx]
        #e = e[-1] if e else 0
        e = int_to_bin(pow(4,e[-1])) if e else '0'*len(num)
        e = [wire_labels['B_'+str(i)] for i in e]
        if len(e) < len(num):
            e = GarbledPadZerosLeft()(e,len(num) - len(e),wire_labels)
        r = [ wire_labels['A_0'] ]*len(e)
        while wire_labels['B_1'] in e:
            #temp = Add()(r, e, wire_labels)
            #temp = [ wire_labels[wl[i].replace('C','A')] for i in temp]
            #print(r,e)
            if GarbledGreaterThanEqualTo()(num, GarbledAdd()(r, e, wire_labels), wire_labels):
                #print('IF')
                out = GarbledAdd()(r, e, wire_labels)
                out = [ wire_labels[wl[i].replace('C','B')] for i in out]
                num = GarbledSubtract()(num, out,wire_labels)
                num = [ wire_labels[wl[i].replace('C','A')] for i in num]
                r = GarbledAdd()(GarbledRightShift()(r, 1, wire_labels), e, wire_labels)
                #print(r)
                r = [ wire_labels[wl[i].replace('C','A')] for i in r]
                #print(r)
            else:
                #print('ELSE')
                r = GarbledRightShift()(r, 1, wire_labels)
            e = GarbledRightShift()(e, 2, wire_labels)
        r = GarbledLeftShift()(r,2, wire_labels)
        return r + [ wire_labels[wl[i].replace('A','C')] for i in x[16:] ]
    
class GarbledCircuit(object):
    def __init__(self):
        pass
        
    def convert_to_float(self,a,two_dimension=True):
        out = np.empty(a.shape[:-1],dtype=np.float)
        wl = dict(zip(self.wire_labels.values(),self.wire_labels.keys()))
        if two_dimension:
            for i in range(a.shape[0]):
                for j in range(a.shape[1]):
                    out[i][j] = bin_to_float(''.join([ wl[k].split('_')[-1] for k in a[i][j] ]))
        else:
            for i in range(a.shape[0]):
                out[i] = bin_to_float(''.join([ wl[k].split('_')[-1] for k in a[i] ]))
                
        return out
    
    def calculate_A_b(self)
        ## Subtract mu_A and mu_B from A_hat and b_hat
        self.A_garbled = np.empty(self.A_hat_garbled.shape,dtype=np.object)
        self.b_garbled = np.empty(self.b_hat_garbled.shape,dtype=np.object)
        #print(self.A_hat_garbled.shape,self.b_hat_garbled.shape,self.mu_A_garbled.shape,self.mu_b_garbled.shape)
        for i in range(self.A_hat_garbled.shape[0]):
            for j in range(self.A_hat_garbled.shape[1]):
                print(i,j)
                #print(self.A_hat_garbled[i][j],self.mu_A_garbled[i][j])
                self.A_garbled[i][j] = np.array(GarbledSubtract()(self.A_hat_garbled[i][j],self.mu_A_garbled[i][j],self.wire_labels))
        
        for i in range(self.b_hat_garbled.shape[0]):
            print(i)
            self.b_garbled[i] = np.array(GarbledSubtract()(self.b_hat_garbled[i],self.mu_b_garbled[i], self.wire_labels))
        
        A = self.convert_to_float(self.A_garbled)
        b = self.convert_to_float(self.b_garbled,False)
        print('A',A.shape,A)
        print('B',b.shape,b)
        
    def calculate_L(self)
        ## Get L using A with algo cholesky
        wl = dict(zip(self.wire_labels.values(), self.wire_labels.keys()))
        self.d = self.A_garbled.shape[0]
        self.L = self.A_garbled.copy()
        for i in range(self.d):
            for j in range(self.d):
                self.L[i][j] = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.L[i][j]])
        #L = np.zeros(A.shape)
        for i in range(self.d-1):
            for j in range(i+1,self.d):
                self.L[i][j] = self.wire_labels['A_0']*24
        for j in range(self.d):
            print(j)
            for k in range(j-1):
                for i in range(j,self.d):
                    ljk = np.array([self.wire_labels[wl[i].replace('A','B')] for i in self.L[j][k]])
                    out = GarbledMultiplication()(self.L[i][k], self.L[j][k],self.wire_labels)
                    out = np.array([self.wire_labels[wl[i].replace('C','B')] for i in out])
                    self.L[i][j] = GarbledSubtract()(self.L[i][j], out, self.wire_labels) ## Need to bring -4 as exponent
                    self.L[i][j] = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.L[i][j]])
        
            self.L[j][j] = GarbledSqrt()(self.L[j][j],self.wire_labels) ## Need to bring -4 as exponent
            self.L[j][j] = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.L[j][j]])
            
            for k in range(j+1,self.d):
                ljj = np.array([self.wire_labels[wl[i].replace('A','B')] for i in self.L[j][j]])
                self.L[k][j] = GarbledDivision()(self.L[k][j], ljj, self.wire_labels) ## Need to bring -4 as exponent
                self.L[k][j] = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.L[k][j]])
                
        LF = self.convert_to_float(self.L)
        print(LF.shape,LF)
        
    def calculate_Y(self):
        
        ## Use back substitution algo to get individual values of Y    : L.T * Y = b
        LT = self.L.transpose(1,0,2)
        self.Y  = np.empty(self.b_garbled.shape,dtype=np.object)
        self.Y[d-1] = GarbledDivision()(self.b_garbled[d-1], LT[d-1][d-1], self.wire_labels)
        self.Y[d-1] = np.array([self.wire_labels[wl[i].replace('C','B')] for i in self.Y[d-1]])
        for i in reversed(range(self.d-1)):
            print(i)
            s = [ self.wire_labels['A_0'] ] * 24
            for j in range(i+1,self.d):
                temp_lij = np.array([self.wire_labels[wl[i].replace('C','A')] for i in LT[i][j]])
                mul = GarbledMultiplication()(temp_lij, self.Y[j], self.wire_labels)
                mul = np.array([self.wire_labels[wl[i].replace('C','B')] for i in mul])
                
                s = GarbledAdd()(s, mul, self.wire_labels)
                s = np.array([self.wire_labels[wl[i].replace('C','A')] for i in s])
                
            s = np.array([self.wire_labels[wl[i].replace('A','B')] for i in s])
            bi = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.b_garbled[i]])
            self.Y[i] = GarbledDivision()(bi, s, self.wire_labels)
            self.Y[i] = np.array([self.wire_labels[wl[i].replace('C','B')] for i in self.Y[i]])
        
        YF = self.convert_to_float(self.Y,False)
        print(YF.shape,YF)
        
    
    def calculate_beta(self):
        ## Use back substitution algo to get individual values of beta : L*beta = Y
        
        beta = np.empty(self.Y.shape,dtype=np.float32)
        beta[0] = GarbledDivision()(self.Y[0], self.L[0][0], self.wire_labels)
        beta[0] = np.array([self.wire_labels[wl[i].replace('C','B')] for i in beta[0]])
        for i in range(1,len(beta)):
            print(i)
            s = [ self.wire_labels['A_0'] ] * 24
            for j in range(i):
                temp_lij = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.L[i][j]])
                mul = GarbledMultiplication()(temp_lij, beta[j], self.wire_labels)
                mul = np.array([self.wire_labels[wl[i].replace('C','B')] for i in mul])
                
                s = GarbledAdd()(s, mul, self.wire_labels)
                s = np.array([self.wire_labels[wl[i].replace('C','A')] for i in s])
                
            s = np.array([self.wire_labels[wl[i].replace('A','B')] for i in s])
            yi = np.array([self.wire_labels[wl[i].replace('B','A')] for i in self.Y[i]])
            beta[i] = GarbledDivision()(yi, s, self.wire_labels)
        
        B = self.convert_to_float(beta)
        print(B.shape,B)
        return beta, self.wire_labels
    
    def execute(self):
        self.calculate_A_b()
        self.calculate_L()
        self.calculate_Y()
        beta,wire_labels = self.calculate_beta()
        return beta,wire_labels

In [13]:
wire_labels = {}
wire_labels['A_0'] = token_hex(4)
wire_labels['A_1'] = token_hex(4)
wire_labels['B_0'] = token_hex(4)
wire_labels['B_1'] = token_hex(4)
wire_labels['C_0'] = token_hex(4)
wire_labels['C_1'] = token_hex(4)
a = '000000000110000011111100'
b = '000000000011000011111100'
a1 = [wire_labels['A_'+i] for i in a]
b1 = [wire_labels['B_'+i] for i in b]

%time ans = GarbledAdd()(a1, b1,wire_labels)
#print(bin_to_int('0000000000110000'),bin_to_int('0000000001100000'),bin_to_int(ans[:16]))
wl = dict(zip(wire_labels.values(),wire_labels.keys()))
''.join([wl[i].split('_')[-1] for i in ans])
print(bin_to_int('0000000000110000'),bin_to_int('0000000001100000'),bin_to_int(''.join([wl[i].split('_')[-1] for i in ans])[:16]))

%time ans = GarbledSubtract()(a1, b1,wire_labels)
print(bin_to_int('0000000000110000'),bin_to_int('0000000001100000'),bin_to_int(''.join([wl[i].split('_')[-1] for i in ans])[:16]))

%time ans = GarbledSubtract()(a1, compute_twos_complement_garbled(b1,wire_labels),wire_labels)
print(bin_to_int('0000000000110000'),bin_to_int('0000000001100000'),bin_to_int(''.join([wl[i].split('_')[-1] for i in ans])[:16]))

%time ans = GarbledMultiplication()(a1, b1,wire_labels)
print(bin_to_float('000000000110000011111100'),bin_to_float('000000000011000011111100'),bin_to_float(''.join([wl[i].split('_')[-1] for i in ans])))

%time ans = GarbledDivision()(a1, b1,wire_labels)
print(bin_to_float('000000000110000011111100'),bin_to_float('000000000011000011111100'),bin_to_float(''.join([wl[i].split('_')[-1] for i in ans])))

%time ans = GarbledSqrt()(a1,wire_labels)
print(bin_to_float('000000000110000011111100'),bin_to_float('000000000011000011111100'),bin_to_float(''.join([wl[i].split('_')[-1] for i in ans])))


CPU times: user 20.1 ms, sys: 0 ns, total: 20.1 ms
Wall time: 20 ms
48 96 144
CPU times: user 46.9 ms, sys: 0 ns, total: 46.9 ms
Wall time: 46.4 ms
48 96 48
CPU times: user 65.5 ms, sys: 304 µs, total: 65.8 ms
Wall time: 64.2 ms
48 96 145
CPU times: user 46 ms, sys: 649 µs, total: 46.7 ms
Wall time: 53 ms
6.0 3.0 18.0
CPU times: user 138 ms, sys: 91 µs, total: 138 ms
Wall time: 137 ms
6.0 3.0 2.0
CPU times: user 249 ms, sys: 86 µs, total: 249 ms
Wall time: 249 ms
6.0 3.0 2.25


In [14]:
class User(object):
    def __init__(self):
        df = pd.read_csv('auto-mpg.csv')
        df = df[df['horsepower'] != '?']
        cols = df.columns.tolist()
        cols.remove('mpg')
        cols.remove('car name')
        self.X = df[cols].values.astype(np.float32)
        self.y = df['mpg'].values.astype(np.float32)
        self.X_train,self.X_test,self.y_train,self.y_test = train_test_split(self.X,self.y,train_size=.8,test_size=.2)
        sc = StandardScaler()
        self.X_train = sc.fit_transform(self.X_train)
        self.X_test = sc.transform(self.X_test)
        self.X_train = np.around(self.X_train,2)
        self.X_test = np.around(self.X_test,2)
        #self.temp = np.around(np.dot(self.X_train.T,self.X_train),2)
        #print(np.around(np.dot(self.X_train.T,self.X_train),2))
    
    def paillier_encrypt(self, A_i, b_i, precision=2):
        A_i_enc = np.zeros(A_i.shape,dtype=np.object)
        for i in range(A_i.shape[0]):
            for j in range(A_i.shape[1]):
                #print(A_i[i][j])
                A_i_enc[i][j] = self.public_key.encrypt(A_i[i][j],precision=precision)
                
        b_i_enc = np.zeros(b_i.shape,dtype=np.object)
        for i in range(b_i.shape[0]):
            b_i_enc[i] = self.public_key.encrypt(b_i[i],precision=precision)
        
        return A_i_enc, b_i_enc
    
    def calculate(self):
        self.c = []
        #print(self.X_train.shape,self.y_train.shape)
        for x_i,y_i in zip(self.X_train,self.y_train):
            #print(x_i, y_i)
            A_i = np.dot(x_i.reshape(len(x_i), 1), x_i.reshape(len(x_i), 1).T)
            b_i = x_i * y_i
            #print(A_i.shape, b_i.shape)
            c_i = self.paillier_encrypt(A_i, b_i, precision=2)
            self.c.append(c_i)
        #c_mu = self.paillier_encrypt(self.mu_A, self.mu_b, precision=2)
        #self.c.append(c_mu)
        return self.c

In [15]:
class CSP(object):
    def __init__(self):
        self.public_key, self.private_key = paillier.generate_paillier_keypair()
        self.mu_Af, self.mu_bf = np.random.randn(7,7), np.random.randn(7)
    
    def paillier_decrypt(self,c_i,precision=2):
        #c = user.calculate_and_send_to_evaluater()
        A_hat, b_hat = np.empty(c_i[0].shape, dtype=np.object), np.empty(c_i[1].shape, dtype=np.object)
        for i in range(c_i[0].shape[0]):
            for j in range(c_i[0].shape[0]):
                A_hat[i][j] = self.private_key.decrypt(c_i[0][i][j])

        for j in range(c_i[1].shape[0]):
                b_hat[j] = self.private_key.decrypt(c_i[1][j])
        
        return A_hat, b_hat
    
    def garble_input(self):
        wire_labels = {}
        wire_labels['A_0'] = token_hex(4)
        wire_labels['A_1'] = token_hex(4)
        wire_labels['B_0'] = token_hex(4)
        wire_labels['B_1'] = token_hex(4)
        wire_labels['C_0'] = token_hex(4)
        wire_labels['C_1'] = token_hex(4)
        
        #self.c_i = self.sock.receive()
        
        #self.A_hat_f, self.b_hat_f = self.paillier_decrypt(self.c_i)
        
        self.A_hat_b = np.empty(self.A_hat_f.shape, dtype = np.object)
        self.b_hat_b = np.empty(self.b_hat_f.shape, dtype = np.object)
        self.mu_A_b = np.empty(self.A_hat_f.shape, dtype = np.object)
        self.mu_b_b = np.empty(self.b_hat_f.shape, dtype = np.object)
        #self.mu_A, self.mu_b = np.empty(self.mu_Af.shape,dtype=np.object), np.empty(self.mu_bf.shape,dtype=np.object)
        for i in range(self.A_hat_f.shape[0]):
            for j in range(self.A_hat_f.shape[1]):
                self.A_hat_b[i][j] = float_to_bin(self.A_hat_f[i][j])
        
        for i in range(self.b_hat_f.shape[0]):
            self.b_hat_b[i] = float_to_bin(self.b_hat_f[i])
        
        for i in range(self.mu_Af.shape[0]):
            for j in range(self.mu_Af.shape[1]):
                self.mu_A_b[i][j] = float_to_bin(self.mu_Af[i][j])
        
        for i in range(self.mu_bf.shape[0]):
            self.mu_b_b[i] = float_to_bin(self.mu_bf[i])
        
        A_hat_garbled = np.empty(self.A_hat_b.shape + (24,),dtype = np.object)
        b_hat_garbled = np.empty(self.b_hat_b.shape + (24,),dtype = np.object)
        mu_A_garbled = np.empty(self.A_hat_b.shape + (24,),dtype = np.object)
        mu_b_garbled = np.empty(self.b_hat_b.shape + (24,),dtype = np.object)
        
        for i in range(self.A_hat_b.shape[0]):
            for j in range(self.A_hat_b.shape[1]):
                A_hat_garbled[i][j] = np.array([(wire_labels['A_0'] if k=='0' else wire_labels['A_1']) for k in self.A_hat_b[i][j]])
        
        for i in range(self.b_hat_b.shape[0]):
            b_hat_garbled[i] = np.array([(wire_labels['A_0'] if k=='0' else wire_labels['A_1']) for k in self.b_hat_b[i]])
            
        for i in range(self.A_hat_b.shape[0]):
            for j in range(self.A_hat_b.shape[1]):
                mu_A_garbled[i][j] = np.array([(wire_labels['B_0'] if k=='0' else wire_labels['B_1']) for k in self.mu_A_b[i][j]])
        
        for i in range(self.b_hat_b.shape[0]):
            mu_b_garbled[i] = np.array([(wire_labels['B_0'] if k=='0' else wire_labels['B_1']) for k in self.mu_b_b[i]])
        
        
        #return self.send_to_evaluator(A_hat_garbled,b_hat_garbled,mu_A_garbled,mu_b_garbled,wire_labels,mu_Af,mu_bf)
        return {
            'A_HAT_GARBLED':A_hat_garbled,
            'B_HAT_GARBLED': b_hat_garbled,
            'MU_A_GARBLED' : mu_A_garbled,
            'MU_B_GARBLED' : mu_b_garbled,
            'WIRE_LABELS'  : wire_labels,
            'PUBLIC KEY'   : self.public_key,
            'MU_AF'        : self.mu_Af,
            'MU_BF'        : self.mu_bf
         }
        
    def send_to_evaluator(self,A_hat_garbled,b_hat_garbled,mu_A_garbled,mu_b_garbled,wire_labels,mu_Af,mu_bf):
        return {
            'A_HAT_GARBLED':A_hat_garbled,
            'B_HAT_GARBLED': b_hat_garbled,
            'MU_A_GARBLED' : mu_A_garbled,
            'MU_B_GARBLED' : mu_b_garbled,
            'WIRE_LABELS'  : wire_labels,
            'PUBLIC KEY'   : self.public_key,
            'MU_AF'        : mu_Af,
            'MU_BF'        : mu_bf
        }

In [28]:
class Evaluator(object):
    def __init__(self):
        #self.A_hat = 
        pass
    
    def receive_c_hat_values(self):
        pass    
    
    def calculate_c_hat(self):
        As,bs = zip(*self.c)
        c_final = sum(As),sum(bs)
        c_final = c_final[0]+self.mu_Af,c_final[1]+self.mu_bf
        return c_final
    
    def send_c_hat_to_csp_for_decryption(self):
        pass
    
    def convert_to_float(self,a,two_dimension=True):
        out = np.empty(a.shape[:-1],dtype=np.float)
        wl = dict(zip(self.wire_labels.values(),self.wire_labels.keys()))
        if two_dimension:
            for i in range(a.shape[0]):
                for j in range(a.shape[1]):
                    out[i][j] = bin_to_float(''.join([ wl[k].split('_')[-1] for k in a[i][j] ]))
        else:
            for i in range(a.shape[0]):
                out[i] = bin_to_float(''.join([ wl[k].split('_')[-1] for k in a[i] ]))
                
        return out
    
    def calculate_A_b(self):
        ## Subtract mu_A and mu_B from A_hat and b_hat
        self.A_garbled = np.empty(self.A_hat_garbled.shape,dtype=np.object)
        self.b_garbled = np.empty(self.b_hat_garbled.shape,dtype=np.object)
        #print(self.A_hat_garbled.shape,self.b_hat_garbled.shape,self.mu_A_garbled.shape,self.mu_b_garbled.shape)
        for i in range(self.A_hat_garbled.shape[0]):
            for j in range(self.A_hat_garbled.shape[1]):
                print(i,j)
                #print(self.A_hat_garbled[i][j],self.mu_A_garbled[i][j])
                self.A_garbled[i][j] = np.array(GarbledSubtract()(self.A_hat_garbled[i][j],self.mu_A_garbled[i][j],self.wire_labels))
        
        for i in range(self.b_hat_garbled.shape[0]):
            print(i)
            self.b_garbled[i] = np.array(GarbledSubtract()(self.b_hat_garbled[i],self.mu_b_garbled[i], self.wire_labels))
        
        A = self.convert_to_float(self.A_garbled)
        b = self.convert_to_float(self.b_garbled,False)
        print('A',A.shape,A)
        print('B',b.shape,b)
        
    def calculate_L(self):
        ## Get L using A with algo cholesky
        wl = dict(zip(self.wire_labels.values(), self.wire_labels.keys()))
        self.d = self.A_garbled.shape[0]
        self.L = self.A_garbled.copy()
        for i in range(self.d):
            for j in range(self.d):
                self.L[i][j] = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.L[i][j]])
        #L = np.zeros(A.shape)
        for i in range(self.d-1):
            for j in range(i+1,self.d):
                self.L[i][j] = [ self.wire_labels['A_0'] ]*24
        for j in range(self.d):
            print(j)
            for k in range(j-1):
                for i in range(j,self.d):
                    ljk = np.array([self.wire_labels[wl[i].replace('A','B')] for i in self.L[j][k]])
                    out = GarbledMultiplication()(self.L[i][k], self.L[j][k],self.wire_labels)
                    out = np.array([self.wire_labels[wl[i].replace('C','B')] for i in out])
                    self.L[i][j] = GarbledSubtract()(self.L[i][j], out, self.wire_labels) ## Need to bring -4 as exponent
                    self.L[i][j] = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.L[i][j]])
        
            self.L[j][j] = GarbledSqrt()(self.L[j][j],self.wire_labels) ## Need to bring -4 as exponent
            self.L[j][j] = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.L[j][j]])
            
            for k in range(j+1,self.d):
                ljj = np.array([self.wire_labels[wl[i].replace('A','B')] for i in self.L[j][j]])
                self.L[k][j] = GarbledDivision()(self.L[k][j], ljj, self.wire_labels) ## Need to bring -4 as exponent
                self.L[k][j] = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.L[k][j]])
                
        LF = self.convert_to_float(self.L)
        print(LF.shape,LF)
        
    def calculate_Y(self):
        
        ## Use back substitution algo to get individual values of Y    : L.T * Y = b
        LT = self.L.transpose(1,0,2)
        self.Y  = np.empty(self.b_garbled.shape,dtype=np.object)
        self.Y[d-1] = GarbledDivision()(self.b_garbled[d-1], LT[d-1][d-1], self.wire_labels)
        self.Y[d-1] = np.array([self.wire_labels[wl[i].replace('C','B')] for i in self.Y[d-1]])
        for i in reversed(range(self.d-1)):
            print(i)
            s = [ self.wire_labels['A_0'] ] * 24
            for j in range(i+1,self.d):
                temp_lij = np.array([self.wire_labels[wl[i].replace('C','A')] for i in LT[i][j]])
                mul = GarbledMultiplication()(temp_lij, self.Y[j], self.wire_labels)
                mul = np.array([self.wire_labels[wl[i].replace('C','B')] for i in mul])
                
                s = GarbledAdd()(s, mul, self.wire_labels)
                s = np.array([self.wire_labels[wl[i].replace('C','A')] for i in s])
                
            s = np.array([self.wire_labels[wl[i].replace('A','B')] for i in s])
            bi = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.b_garbled[i]])
            self.Y[i] = GarbledDivision()(bi, s, self.wire_labels)
            self.Y[i] = np.array([self.wire_labels[wl[i].replace('C','B')] for i in self.Y[i]])
        
        YF = self.convert_to_float(self.Y,False)
        print(YF.shape,YF)
        
    
    def calculate_beta(self):
        ## Use back substitution algo to get individual values of beta : L*beta = Y
        
        beta = np.empty(self.Y.shape,dtype=np.float32)
        beta[0] = GarbledDivision()(self.Y[0], self.L[0][0], self.wire_labels)
        beta[0] = np.array([self.wire_labels[wl[i].replace('C','B')] for i in beta[0]])
        for i in range(1,len(beta)):
            print(i)
            s = [ self.wire_labels['A_0'] ] * 24
            for j in range(i):
                temp_lij = np.array([self.wire_labels[wl[i].replace('C','A')] for i in self.L[i][j]])
                mul = GarbledMultiplication()(temp_lij, beta[j], self.wire_labels)
                mul = np.array([self.wire_labels[wl[i].replace('C','B')] for i in mul])
                
                s = GarbledAdd()(s, mul, self.wire_labels)
                s = np.array([self.wire_labels[wl[i].replace('C','A')] for i in s])
                
            s = np.array([self.wire_labels[wl[i].replace('A','B')] for i in s])
            yi = np.array([self.wire_labels[wl[i].replace('B','A')] for i in self.Y[i]])
            beta[i] = GarbledDivision()(yi, s, self.wire_labels)
        
        B = self.convert_to_float(beta)
        print(B.shape,B)
        return beta, self.wire_labels

In [17]:
user = User()

In [18]:
csp = CSP()

In [19]:
evaluator = Evaluator()

In [20]:
#user.public_key = csp.public_key
#%time c = user.calculate()

In [21]:
#evaluator.c = c

evaluator.mu_Af = csp.mu_Af
evaluator.mu_bf = csp.mu_bf
#%time c_final = evaluator.calculate_c_hat()

In [22]:
#csp.c_i = c_final
csp.A_hat_f = np.dot(user.X_train.T,user.X_train)
csp.b_hat_f = np.dot(user.X_train.T, user.y_train)
%time d = csp.garble_input()

CPU times: user 22.4 ms, sys: 0 ns, total: 22.4 ms
Wall time: 22.1 ms


In [23]:
evaluator.A_hat_garbled = d['A_HAT_GARBLED']
evaluator.b_hat_garbled = d['B_HAT_GARBLED']
evaluator.mu_A_garbled = d['MU_A_GARBLED']
evaluator.mu_b_garbled = d['MU_B_GARBLED']
evaluator.wire_labels = d['WIRE_LABELS']

print(evaluator.A_hat_garbled.shape)
print(evaluator.b_hat_garbled.shape)
print(evaluator.mu_A_garbled.shape)
print(evaluator.mu_b_garbled.shape)


(7, 7, 24)
(7, 24)
(7, 7, 24)
(7, 24)


In [24]:
#evaluator.A_hat_garbled[0][0],evaluator.b_hat_garbled[0],evaluator.mu_A_garbled[0][0],evaluator.mu_b_garbled[0]

In [25]:
evaluator.wire_labels

{'A_0': 'dbc03031',
 'A_1': '5b90a36d',
 'B_0': 'ced4a834',
 'B_1': '35ad0ceb',
 'C_0': '25946e03',
 'C_1': '1fcb8d24'}

In [26]:
%time evaluator.calculate_A_b()

0 0
0 1
0 2
0 3
0 4
0 5
0 6
1 0
1 1
1 2
1 3
1 4
1 5
1 6
2 0
2 1
2 2
2 3
2 4
2 5
2 6
3 0
3 1
3 2
3 3
3 4
3 5
3 6
4 0
4 1
4 2
4 3
4 4
4 5
4 6
5 0
5 1
5 2
5 3
5 4
5 5
5 6
6 0
6 1
6 2
6 3
6 4
6 5
6 6
0
1
2
3
4
5
6
A (7, 7) [[ 315.      298.0625  260.8125  283.5625 -412.5    -236.375  -435.    ]
 [ 297.5625  313.125   278.3125  293.25   -422.125  -245.125  -451.5625]
 [ 264.3125  284.875   312.      269.5    -475.8125 -391.5625 -399.0625]
 [ 278.375   296.5     268.125   311.0625 -391.     -226.5    -443.75  ]
 [-412.25   -425.6875 -476.4375 -389.1875  313.8125  101.9375   68.9375]
 [-236.6875 -237.375  -390.     -225.25    102.4375  312.8125   64.4375]
 [-435.5    -451.375  -399.5    -444.25     70.0625   64.4375  309.375 ]]
B (7,) [ 120.375    71.25    134.625    14.     1108.     1440.8125 1428.3125]
CPU times: user 5min 45s, sys: 725 ms, total: 5min 46s
Wall time: 5min 46s


In [ ]:
%time evaluator.calculate_L()

0


In [ ]:
%time evaluator.calculate_Y()

In [ ]:
%time evaluator.calculate_beta()

In [85]:
# https://codereview.stackexchange.com/questions/174336/rsa-algorithm-implementation-in-python-3
import math

def modinv(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def coprimes(a, phi):
    l = []
    for x in range(2, a):
        if math.gcd(a, x) == 1 and modinv(x,phi) != None:
            l.append(x)
    for x in l:
        if x == modinv(x,phi):
            l.remove(x)
    return l

def RSA():
    p, q = 5, 7
    n=p*q
    phi=(p-1)*(q-1)
    #print(coprimes(phi, phi))
    e = np.random.choice(coprimes(phi, phi))
    d=modinv(e,phi)
    return d,n,e    

In [93]:
# ALICE

muAs = [np.random.randn(5,5) for i in range(5)]
muBs = [np.random.randn(5) for i in range(5)]

d,N,e = RSA()
x = np.random.choice(1000,5)
print(d, N, e, x)

7 35 7 [734 333 576 869 748]


In [94]:
## BOB
k,b = np.random.choice(100), np.random.choice(5)

v = (x[b] + pow(k,e)) % N
print(k,v,b)

52 33 0


In [95]:
ks = [(pow((v-i),d)) % N for i in x]
ks

[11, 17, 4, 15, 7]

In [89]:
m_primes = [muA+i for muA,i in zip(muAs,ks)]

In [90]:
m_primes[b] - k

array([[-67.98705941, -68.01985262, -69.02921071, -71.08564611,
        -69.41600979],
       [-69.4143883 , -70.27090559, -70.10558591, -68.46203219,
        -70.85641725],
       [-70.28717842, -69.55270803, -66.95859012, -69.30854727,
        -70.26980171],
       [-68.76590253, -70.63873734, -68.73215956, -68.77670277,
        -67.74994346],
       [-71.07292182, -69.61929012, -68.1048682 , -69.35772143,
        -70.39861851]])

In [91]:
muAs[b]

array([[ 1.01294059,  0.98014738, -0.02921071, -2.08564611, -0.41600979],
       [-0.4143883 , -1.27090559, -1.10558591,  0.53796781, -1.85641725],
       [-1.28717842, -0.55270803,  2.04140988, -0.30854727, -1.26980171],
       [ 0.23409747, -1.63873734,  0.26784044,  0.22329723,  1.25005654],
       [-2.07292182, -0.61929012,  0.8951318 , -0.35772143, -1.39861851]])